In [0]:
# HELLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO 👋












# Chris: What else we gotta do?
# Aggregation #5
# And running it to make sure the transformatiions work
# Ok gotcha, which would be easier or better, top repos or org vs non org
# Well we could tie in both, so we can get the top repos, and check if they belong to an org
# ok

##### You are tasked with the following aggregations:

- Data aggregated by type of GitHub event per hour
- PushEvent data aggregated by ref type – whether the commit is on the main branch
- Breakdown of events by type and number of commits per event
- User activity should be aggregated so that a filterable chart can be populated with breakdowns of user activity by week or month.
- Breakdown of activity by project – find a unique use case
- Challenge: Based on the commit messages – breakdown the events by language

In [0]:
# TO DO:
# Create a date dimension table (Javi: I know you can use one of the from_unix or from_epoch to extract the month, day, year in integer form already using month() and such)
# Make aggregation fact tables based on project goals?
    # Follow gold layer erd basically

# Aggregations we need to take into consideration:
    # 1. Data aggregated by type of GitHub event per hour -> will use that date dimension table and event_type table

    # 2. PushEvent data aggregated by ref type – whether the commit is on the main branch -> Will use push event dim table and repo dim table

    # 3. Breakdown of events by type and number of commits per event -> Will use fact event table (for event type) and push event table for commit count

    # 4. User activity should be aggregated so that a filterable chart can be populated with breakdowns of user activity by week or month.
    # -> Idea: Use the date dimension table that will be made to filter by month to perform some sort of breakdowns.
    # Ex: Event count per month/week, number of commits per month/week (could be used to analyze slow and peak seasons of github?)

    # 5. Breakdown of activity by project – find a unique use case
    # -> Will use event fact table and it's pertaining event information dim table.

# Javi: Do we really need to make any different tables besides the date?
# Chris: Honestly not really but I'm worried that won't fulfill requirements, maybe ask Jordan?

# Type of gold layer fact tables we could make:
    # Perhaps the aggregations themselves could be the tables we make for our gold layer
    # Ex: For the 1st aggregation, we can make a fact user activity with the event type and hour columns

In [0]:
# When reading in all the files, we can specify the schema to speed up reads:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, BooleanType

# FACT EVENTS
fact_events_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("event_type", StringType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("actor_id", LongType(), True),
    StructField("repo_id", LongType(), True),
    StructField("organization_id", LongType(), True)
])


# PULL REQUESTS
fact_pull_request_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("action", StringType(), True),
    StructField("pr_closed_at", StringType(), True),
    StructField("pr_additions", LongType(), True),
    StructField("pr_deletions", LongType(), True),
    StructField("pr_changed_files", LongType(), True),
    StructField("pr_commits", LongType(), True),
    StructField("pr_base_ref", StringType(), True),
    StructField("pr_head_ref", StringType(), True),
    StructField("pr_merged", BooleanType(), True)
])

# ISSUE
fact_issue_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("action", StringType(), True),
    StructField("issue_closed_at", StringType(), True)
])

# PUSH 
fact_push_event_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("commit_count", LongType(), True),
    StructField("distinct_size", LongType(), True),
    StructField("ref", StringType(), True)
])

# CREATE
fact_create_event_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("ref", StringType(), True),
    StructField("ref_type", StringType(), True)
])

# DELETE
fact_delete_event_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("ref", StringType(), True),
    StructField("ref_type", StringType(), True)
])

# FORK
fact_fork_event_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("forkee_id", LongType(), True),
    StructField("forkee_name", StringType(), True),
    StructField("forkee_language", StringType(), True)
])

# MEMBER
fact_member_event_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("action", StringType(), True),
    StructField("member_id", LongType(), True),
    StructField("member_login", StringType(), True)
])

# ISSUE COMMENT
fact_issue_comment_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("comment_id", LongType(), True),
    StructField("issue_number", IntegerType(), True)
])

# REPO
repo_lookup_schema = StructType([
    StructField("repo_id", LongType(), False),
    StructField("repo_name", StringType(), True),
    StructField("repo_language", StringType(), True),
    StructField("repo_default_branch", StringType(), True),
    StructField("repo_created_at", StringType(), True),
    StructField("repo_is_fork", BooleanType(), True),
    StructField("repo_is_private", BooleanType(), True)
])

# ORG
org_lookup_schema = StructType([
    StructField("organization_id", LongType(), False),
    StructField("org_login", StringType(), True)
])

# ACTOR
actor_lookup_schema = StructType([
    StructField("actor_id", LongType(), False),
    StructField("actor_login", StringType(), True)
])



#### Fact & Dimension Tables

In [0]:
# Reading in silver layer data:
# Every event should be read separately?
# If so, it'll be done like this:

# Base path
base_path = "abfss://team3-project2@20251124eyproject2.dfs.core.windows.net/silver"

# Reading in the files for the different tables
# Separated for readability
# NOTE: The final path wasn't entered, that way we can read in we can customize what files and how many files are read in
#       Layout is just base layout that all files are read in
# Chris: That makes sense for bronze, but theres no file level customization for silver? You need to read the whole folder (I added folder names).
# Chris: Also, I changed names

# FACT EVENTS
# Based on Jordan's notes, since we're using fact events multiple times, it's good to cache it
fact_events = spark.read.schema(fact_events_schema).parquet(f"{base_path}/fact_events/").cache()

# PULL REQUESTS
fact_pull_request = spark.read.schema(fact_pull_request_schema).parquet(f"{base_path}/fact_pull_request/")

# ISSUE
fact_issue = spark.read.schema(fact_issue_schema).parquet(f"{base_path}/fact_issue/")

# PUSH 
fact_push_event = spark.read.schema(fact_push_event_schema).parquet(f"{base_path}/fact_push_event/")

# CREATE
fact_create_event = spark.read.schema(fact_create_event_schema).parquet(f"{base_path}/fact_create_event/")

# DELETE
fact_delete_event = spark.read.schema(fact_delete_event_schema).parquet(f"{base_path}/fact_delete_event/")

# FORK
fact_fork_event = spark.read.schema(fact_fork_event_schema).parquet(f"{base_path}/fact_fork_event/")

# MEMBER
fact_member_event = spark.read.schema(fact_member_event_schema).parquet(f"{base_path}/fact_member_event/")

# ISSUE COMMENT
fact_issue_comment = spark.read.schema(fact_issue_comment_schema).parquet(f"{base_path}/fact_issue_comment/")

# REPO
dim_repo = spark.read.schema(repo_lookup_schema).parquet(f"{base_path}/repo_lookup/")

# ORG
dim_org = spark.read.schema(org_lookup_schema).parquet(f"{base_path}/org_lookup/")

# ACTOR
dim_actor = spark.read.schema(actor_lookup_schema).parquet(f"{base_path}/actor_lookup/")

# Please make sure the code is fine, or if theres a better way of doing this!
# Looks good to me



# Chris: We don't need schema for parquet files? Parquet files store schema in file metadata already
# Chris: I would literally just do: fact_events_df = spark.read.parquet(f"{base_path}/fact_events/"), thats it, nothing else
# Stanley: actually yea we dont, google says it may slightly improve performance but thats all
# Chris: Lol nm Jordan said we needed em
# Javi: Nice!

#### Aggregations

In [0]:
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofweek, col, count, when, sum as spark_sum, weekofyear, countDistinct, broadcast

# Core Aggregations - FOLKS THIS IS A ROUGH DRAFT FEEL FREE TO CHANGE ANYTHING, ALSO PLEASE LOOK AT MY SUGGESTIONS - BASICALLY I DON'T THINK SCHEMA IS NEEDED - CHRIS

# Create dim_date (needed for everything else)
dim_date = (
    fact_events
    .select("created_at")
    .withColumn("year", year("created_at"))
    .withColumn("month", month("created_at"))
    .withColumn("day", dayofmonth("created_at"))
    .withColumn("hour", hour("created_at"))
    .withColumn("day_of_week", dayofweek("created_at"))
    .distinct()
)

# Aggregation 1: Events by type per hour
gold_events_by_type_hour = (
    fact_events
    .withColumn("year", year("created_at"))
    .withColumn("hour", hour("created_at"))
    .groupBy("event_type", "year", "hour")
    .agg(count("*").alias("event_count"))
    .orderBy("event_type", "year", "hour")
)

# Aggregation 2: Push by branch 
gold_push_by_branch = (
    fact_push_event
    .withColumn("branch_type", 
        when(
            (col("ref").endswith("/main")) | (col("ref").endswith("/master")), 
            "main"
        ).otherwise("other")
    )
    .groupBy("branch_type")
    .agg(
        count("*").alias("push_count"),
        spark_sum("commit_count").alias("total_commits")
    )
)

# Javi: They look good Chris!

# Aggregation 3: Events by type and number of commits
gold_events_by_commits = (
    fact_push_event
    .groupBy("commit_count")
    .agg(count("*").alias("push_count"))
    .orderBy("commit_count")
)

# Aggregation 4: Analyzing event count and commit count by week/month
# Javi: Will be doing it by week, gives more granularity.
# You can make a really nice line graph with this

gold_event_count_count_week = (
    fact_events
    .withColumn("year", year("created_at"))
    .withColumn("week", weekofyear("created_at"))
    .groupBy("actor_id", "year", "week")
    .agg(count('*').alias('event_count'))
    .orderBy("year", "week")
)

gold_commit_count_week = (
    fact_events.select("event_id", "created_at")
    .join(
        broadcast(fact_push_event.select("event_id", "commit_count")), # fact_push_event is 104M rows, might be too big
        # Nvm, the threshold is 8GB, but preferred 10 MB
        on='event_id',
        how='inner'
    )
    .withColumn("year", year("created_at"))
    .withColumn('week', weekofyear('created_at'))
    .groupBy("year", "week")
    .agg(spark_sum("commit_count").alias('total_commits'))
    .orderBy("year", "week")
)

# Aggregation 5: Breakdown of activity by project – find a unique use case
# Idea: We can do issue event activity. How fast they were resolved. The amount of commits done, 
# Idea No.2: Find projects that appear multiple times. Get their count. Top 10 projects by count of the year 2015
    # What's the most prevalent event? 
    # Perhaps a project is open source, does it belong to any organization?
        # How does a project belonging to an organization differ in the amount of commits say to a project that doesnt

# Aggregation 5: Top repos with org info
gold_top_repos = (
    fact_events
    .join(dim_repo, on="repo_id", how="left")
    .join(dim_org, on="organization_id", how="left")
    .groupBy("repo_id", "repo_name", "organization_id", "org_login")
    .agg(
        count("*").alias("total_events"),
        countDistinct("event_type").alias("event_diversity"),
        countDistinct("actor_id").alias("unique_contributors")
    )
    .orderBy(col("total_events").desc())
)

In [0]:
# Test all aggregations
print("=== dim_date ===")
display(dim_date.limit(10))

print("=== Agg 1: Events by type per hour ===")
display(gold_events_by_type_hour)

print("=== Agg 2: Push by branch ===")
display(gold_push_by_branch)

print("=== Agg 3: Events by commits ===")
display(gold_events_by_commits.limit(20))

print("=== Agg 4a: User activity by week ===")
display(gold_event_count_count_week.limit(20))

print("=== Agg 4b: Commits by week ===")
display(gold_commit_count_week)

print("=== Agg 5: Top repos ===")
display(gold_top_repos.limit(100))

=== dim_date ===


created_at,year,month,day,hour,day_of_week
2015-09-30T15:05:09.000Z,2015,9,30,15,4
2015-09-30T15:05:44.000Z,2015,9,30,15,4
2015-09-30T15:06:03.000Z,2015,9,30,15,4
2015-09-30T15:26:58.000Z,2015,9,30,15,4
2015-09-30T15:56:19.000Z,2015,9,30,15,4
2015-10-07T15:26:08.000Z,2015,10,7,15,4
2015-10-07T15:34:49.000Z,2015,10,7,15,4
2015-10-07T15:43:07.000Z,2015,10,7,15,4
2015-10-07T15:53:12.000Z,2015,10,7,15,4
2015-10-13T14:16:36.000Z,2015,10,13,14,3


=== Agg 1: Events by type per hour ===


event_type,hour,event_count
CommitCommentEvent,0,40136
CommitCommentEvent,1,34958
CommitCommentEvent,2,31918
CommitCommentEvent,3,29749
CommitCommentEvent,4,27418
CommitCommentEvent,5,28761
CommitCommentEvent,6,34620
CommitCommentEvent,7,42344
CommitCommentEvent,8,49612
CommitCommentEvent,9,54322


=== Agg 2: Push by branch ===


...

actor_id,week,event_count
8956796,1,51
785405,1,5
2482827,1,25
6232704,1,120
1503233,1,19
6187125,1,65
9049806,1,1
97068,1,8
899884,1,12
245998,1,5


=== Agg 4b: Commits by week ===


week,total_commits
1,3031381
2,6560564
3,7223359
4,8841514
5,7987734
6,7635742
7,7410799
8,8032498
9,8167613
10,8913169


=== Agg 5: Top repos ===


repo_id,repo_name,organization_id,org_login,total_events,event_diversity,unique_contributors
30185757,null,null,null,659546,5,2
21481110,null,null,null,509779,3,32
43155190,null,null,null,310732,2,2
6058234,null,2033545,sakai-mirror,256518,1,1
23808509,null,4982125,Door43,226905,2,4
17165658,apache/spark,47359,apache,223163,9,8897
35637132,null,null,null,189974,3,1
32394848,null,null,null,151983,6,5
7691631,docker/docker,5429470,docker,132949,13,19456
20580498,kubernetes/kubernetes,13629408,kubernetes,129689,13,4292


#### Writing to gold layer

In [0]:
# Javi: Would we be saving these aggregations to our gold layer? Probably right?

# Stanley: I think so. Where would we be creating our visualizations though?
# Javi: Inside databricks? We can use the display function when we've made the aggregation tables and make some quick visualizations

path_to_gold = "abfss://team3-project2@20251124eyproject2.dfs.core.windows.net/gold/"

# Javi: I wonder if we just resave all the tables we got our just the aggregation tables?
# Stanley: I think just the aggregation tables, the other fact and dimension tables can be easily recreated with the parquet files


# EXAMPLE
tables = {
    "dim_date": dim_date,
    "gold_events_by_type_hour": gold_events_by_type_hour,
    "gold_push_by_branch": gold_push_by_branch,
    "gold_events_by_commits": gold_events_by_commits,
    "gold_event_count_count_week": gold_event_count_count_week,
    "gold_commit_count_week": gold_commit_count_week,
    "gold_top_repos": gold_top_repos,
}

# No need to partition right?
# I think we still need to partition when we write to gold/ but it should be really easy since we already have the sizes
# as parquet
# Stanley: Looking online, and it seems we can just coalesce and write to disk
for name, df in tables.items():
    df.write.mode("overwrite").parquet(path_to_gold + name + "/")
    print(f"Wrote {name}")

#### Visualizations

In [0]:
# Never used databricks visualizations, will run some tests here
# Stanley: should be really easy, just ran some code on free version

# display(agg df), then press the + symbol and add a visual
# Javi: Yea it's this easy! The display function is how we can do visualizations in databricks!
